# 🎭 Ideal Anime Character Match Predictor

**A personality-based anime character matching system with beautiful results!**

### ✨ Features:
- **Interactive Scenarios**: Answer personality-based questions
- **Smart Algorithm**: Multi-dimensional trait compatibility analysis
- **Beautiful Results**: Stunning web page with character images
- **Top Matches**: Your top 3 ideal matches with detailed stats
- **LangChain Integration**: Optional AI-generated scenarios

### 🚀 Quick Start:
1. Run all cells in order (Ctrl+Shift+Enter)
2. Answer 7 personality scenarios
3. Get beautiful results in your browser!

---

In [2]:
# 📦 Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import random
import webbrowser
import tempfile
import os
import json
from datetime import datetime
import time

# Optional LangChain imports (for AI scenario generation)
try:
    from langchain_ollama import OllamaLLM
    from langchain.prompts import PromptTemplate
    from langchain_core.output_parsers import JsonOutputParser
    from pydantic import BaseModel, Field
    from typing import Dict, List
    LANGCHAIN_AVAILABLE = True
    print("✅ LangChain available - AI scenario generation enabled")
except ImportError:
    LANGCHAIN_AVAILABLE = False
    print("⚠️  LangChain not available - using pre-made scenarios only")

print("📚 All libraries imported successfully!")

✅ LangChain available - AI scenario generation enabled
📚 All libraries imported successfully!


In [3]:
# 📊 Load Character Data
DATA_PATH = "./data.csv"
try:
    df = pd.read_csv(DATA_PATH)
    print(f"✅ Loaded data for {len(df)} characters")
except FileNotFoundError:
    print(f"❌ Error: {DATA_PATH} not found. Make sure 'data.csv' is in the same directory.")
    df = pd.DataFrame({'name': ['Dummy'], 'summary': ['Dummy summary'], 'Trait1': [50], 'Trait2': [50]})

NAME_COL = "name"
SUMMARY_COL = "summary"
trait_cols = df.select_dtypes(include=np.number).columns.tolist()

print(f"📈 Personality traits loaded: {len(trait_cols)} traits")
print(f"🎭 Character names: {df[NAME_COL].tolist()[:5]}{'...' if len(df) > 5 else ''}")

# Normalize and center traits
scaler = MinMaxScaler()
traits_norm = pd.DataFrame(scaler.fit_transform(df[trait_cols]) - 0.5, columns=trait_cols, index=df.index)
print("✅ Traits normalized and centered for analysis")

✅ Loaded data for 25 characters
📈 Personality traits loaded: 20 traits
🎭 Character names: ['Asuna Yuuki', 'Rem', 'Hinata Hyuga', 'Tsunade Senju', 'Nico Robin']...
✅ Traits normalized and centered for analysis


In [4]:
# 🎭 Pre-made Personality Scenarios
ALL_SCENARIOS = [
    {
        "scenario_question": "Festival day in your city. Your partner prefers to:",
        "option_a": "A) Mingle with crowds and meet new people",
        "option_b": "B) Find a quiet spot to watch fireworks together",
        "vector_a": {"Social Acuity": 2.0, "Optimism": 1.0},
        "vector_b": {"Independence": 1.5, "Emotional Stability": 1.0}
    },
    {
        "scenario_question": "A friend is wrongly accused of a crime. Your partner:",
        "option_a": "A) Immediately supports them with unwavering loyalty",
        "option_b": "B) Suggests examining evidence objectively first",
        "vector_a": {"Loyalty": 2.0, "Empathy": 1.5},
        "vector_b": {"Cynicism": 1.5, "Intellect": 1.0}
    },
    {
        "scenario_question": "Sudden magical threat appears. Your partner:",
        "option_a": "A) Acts on instinct, unleashing full power immediately",
        "option_b": "B) Takes defensive stance, analyzes weaknesses first",
        "vector_a": {"Combat Prowess": 2.0, "Impulsiveness": 1.5},
        "vector_b": {"Discipline": 2.0, "Intellect": 1.5}
    },
    {
        "scenario_question": "Ancient library discovered. Your partner wants to:",
        "option_a": "A) Dive into forbidden sections for powerful knowledge",
        "option_b": "B) Systematically catalog everything for preservation",
        "vector_a": {"Ambition": 2.0, "Impulsiveness": 1.0},
        "vector_b": {"Discipline": 2.0, "Altruism": 1.5}
    },
    {
        "scenario_question": "After a heartbreaking setback, your partner:",
        "option_a": "A) Shares feelings openly, seeking comfort",
        "option_b": "B) Retreats privately to process grief alone",
        "vector_a": {"Empathy": 1.0, "Nurturance": 1.0},
        "vector_b": {"Resilience": 2.0, "Independence": 1.5}
    },
    {
        "scenario_question": "You discover their secret talent. They:",
        "option_a": "A) Light up and eagerly share everything about it",
        "option_b": "B) Blush and downplay it as unimpressive",
        "vector_a": {"Self-Esteem": 2.0, "Optimism": 1.5},
        "vector_b": {"Self-Esteem": -1.5, "Empathy": 1.0}
    },
    {
        "scenario_question": "Group project disagreement. Your partner:",
        "option_a": "A) Takes charge with a clear plan",
        "option_b": "B) Listens to all sides, suggests compromise",
        "vector_a": {"Assertiveness": 2.0, "Ambition": 1.5},
        "vector_b": {"Empathy": 2.0, "Adaptability": 1.5}
    },
    {
        "scenario_question": "Dangerous mission opportunity arises. Your partner:",
        "option_a": "A) Volunteers immediately for the challenge",
        "option_b": "B) Carefully weighs risks before deciding",
        "vector_a": {"Combat Prowess": 1.5, "Impulsiveness": 2.0},
        "vector_b": {"Intellect": 2.0, "Emotional Stability": 1.5}
    },
    {
        "scenario_question": "Someone needs help but can't pay. Your partner:",
        "option_a": "A) Helps immediately without expecting anything",
        "option_b": "B) Explains they need to prioritize paying clients",
        "vector_a": {"Altruism": 2.0, "Nurturance": 1.5},
        "vector_b": {"Cynicism": 1.5, "Discipline": 1.0}
    },
    {
        "scenario_question": "Training session with others. Your partner:",
        "option_a": "A) Pushes themselves to outperform everyone",
        "option_b": "B) Focuses on helping weaker teammates improve",
        "vector_a": {"Ambition": 2.0, "Assertiveness": 1.5},
        "vector_b": {"Nurturance": 2.0, "Empathy": 1.5}
    },
    {
        "scenario_question": "Plans fall through last minute. Your partner:",
        "option_a": "A) Quickly improvises something fun instead",
        "option_b": "B) Gets frustrated and needs time to adjust",
        "vector_a": {"Adaptability": 2.0, "Optimism": 1.5},
        "vector_b": {"Emotional Stability": -1.0, "Perseverance": 1.0}
    },
    {
        "scenario_question": "Facing a moral dilemma. Your partner:",
        "option_a": "A) Does what feels right in their heart",
        "option_b": "B) Analyzes consequences logically first",
        "vector_a": {"Empathy": 2.0, "Impulsiveness": 1.0},
        "vector_b": {"Intellect": 2.0, "Discipline": 1.5}
    }
]

print(f"🎭 {len(ALL_SCENARIOS)} personality scenarios ready!")

🎭 12 personality scenarios ready!


In [5]:
# 🖼️ Character Image URLs (for beautiful results)
CHARACTER_IMAGES = {
    "Asuna Yuuki": "https://static.wikia.nocookie.net/swordartonline/images/f/fe/Asuna_SAO.png",
    "C.C.": "https://static.wikia.nocookie.net/codegeass/images/c/cc/C.C._main.jpg",
    "Emilia": "https://static.wikia.nocookie.net/rezero/images/6/6d/Emilia_-_Re_Zero_Anime_BD.png",
    "Erza Scarlet": "https://static.wikia.nocookie.net/fairytail/images/8/88/Erza_X791.png",
    "Hinata Hyūga": "https://static.wikia.nocookie.net/naruto/images/1/15/Hinata_Part_II.png",
    "Holo": "https://static.wikia.nocookie.net/spiceandwolf/images/a/a4/Holo_infobox.png",
    "Kurisu Makise": "https://static.wikia.nocookie.net/steins-gate/images/0/04/Kurisu_Makise.png",
    "Mai Sakurajima": "https://static.wikia.nocookie.net/aobuta/images/6/65/Mai_Sakurajima_Anime.png",
    "Maki Zenin": "https://static.wikia.nocookie.net/jujutsu-kaisen/images/8/82/Maki_Zenin_anime.png",
    "Marin Kitagawa": "https://static.wikia.nocookie.net/sono-bisque-doll-wa-koi-wo-suru/images/2/25/Marin_Kitagawa_anime.png",
    "Mitsuri Kanroji": "https://static.wikia.nocookie.net/kimetsu-no-yaiba/images/a/a2/Mitsuri_anime.png",
    "Nezuko Kamado": "https://static.wikia.nocookie.net/kimetsu-no-yaiba/images/7/75/Nezuko_anime_design.png",
    "Nico Robin": "https://static.wikia.nocookie.net/onepiece/images/b/b3/Nico_Robin_Anime_Post_Timeskip_Infobox.png",
    "Nobara Kugisaki": "https://static.wikia.nocookie.net/jujutsu-kaisen/images/1/15/Nobara_Kugisaki_anime.png",
    "Power": "https://static.wikia.nocookie.net/chainsaw-man/images/7/7a/Power_anime_design.png",
    "Rem": "https://static.wikia.nocookie.net/rezero/images/6/6b/Rem_Anime.png",
    "Rias Gremory": "https://static.wikia.nocookie.net/highschooldxd/images/f/f1/Rias_Gremory.png",
    "Shinobu Kocho": "https://static.wikia.nocookie.net/kimetsu-no-yaiba/images/1/14/Shinobu_anime.png",
    "Shinobu Oshino": "https://static.wikia.nocookie.net/bakemonogatari1645/images/3/3b/Shinobu_Oshino.png",
    "Tohru": "https://static.wikia.nocookie.net/kobayashisdragonmaid/images/d/d5/Tohru_anime.png",
    "Tsunade": "https://static.wikia.nocookie.net/naruto/images/b/b3/Tsunade_infobox2.png",
    "Yor Forger": "https://static.wikia.nocookie.net/spy-x-family6458/images/e/e2/Yor_Forger_Anime.png",
    "Yoruichi Shihōin": "https://static.wikia.nocookie.net/bleach/images/f/f1/Yoruichi_mugshot.png",
    "Yukino Yukinoshita": "https://static.wikia.nocookie.net/yahari/images/c/c8/Yukino_S3.png",
    "Zero Two": "https://static.wikia.nocookie.net/darling-in-the-franxx/images/a/a1/Zero_Two_anime.png"
}

print(f"🖼️  Character images ready for {len(CHARACTER_IMAGES)} characters!")

🖼️  Character images ready for 25 characters!


In [6]:
# 🎯 Core Functions

def get_scenario(history=None):
    """Get a scenario from the available list, avoiding repeats"""
    if history is None:
        history = []
    
    # Filter out already used scenarios
    available_scenarios = [s for s in ALL_SCENARIOS if s['scenario_question'] not in history]
    
    if not available_scenarios:
        print("🔄 All scenarios used, recycling...")
        available_scenarios = ALL_SCENARIOS
    
    selected = random.choice(available_scenarios)
    return selected

def ask_scenario_question(scenario_data, question_num=1, total_questions=7):
    """Present scenario and get user choice"""
    print("\n" + "═" * 70)
    print(f"✨ SCENARIO {question_num}/{total_questions} ✨")
    print("═" * 70)
    print(f"🎭 {scenario_data['scenario_question']}")
    print()
    print(f"🔵 A) {scenario_data['option_a']}")
    print(f"🔴 B) {scenario_data['option_b']}")
    print()
    print("Choose your preference:")
    print("  💪 [1] Strongly A    👍 [2] Prefer A    🤷 [3] Neutral    👍 [4] Prefer B    💪 [5] Strongly B")
    print("═" * 70)
    
    choice_map = {
        '1': (1.0, 'a'), '2': (0.5, 'a'), '3': (0.0, 'a'),
        '4': (0.5, 'b'), '5': (1.0, 'b')
    }
    
    while True:
        choice = input("Enter your choice (1-5): ").strip()
        if choice in choice_map:
            multiplier, choice_letter = choice_map[choice]
            choice_text = {
                (1.0, "a"): "💪 Strongly prefer A",
                (0.5, "a"): "👍 Prefer A", 
                (0.0, "a"): "🤷 Neutral",
                (0.5, "b"): "👍 Prefer B",
                (1.0, "b"): "💪 Strongly prefer B"
            }
            print(f"✅ You chose: {choice_text.get((multiplier, choice_letter), 'Unknown')}")
            return (multiplier, choice_letter)
        print("❌ Please enter a number from 1-5")

print("🎯 Core functions ready!")

🎯 Core functions ready!


In [7]:
# 🌟 Beautiful Results Page Generator

def create_beautiful_results_html(top_matches, user_profile):
    """Create stunning HTML results page with character images"""
    
    # Get top 3 character details
    top_3_details = []
    for i, (name, score, char_data) in enumerate(top_matches[:3]):
        top_3_details.append({
            'rank': i + 1,
            'name': name,
            'score': score,
            'image': CHARACTER_IMAGES.get(name, ""),
            'summary': char_data.get('summary', ''),
            'personality_score': char_data.get('personality_score', 0)
        })
    
    html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🎭 Your Ideal Match Results</title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600;700;800&display=swap');
        
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Poppins', sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            color: #333;
            animation: gradientShift 12s ease infinite;
        }}
        
        @keyframes gradientShift {{
            0%, 100% {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); }}
            33% {{ background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); }}
            66% {{ background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); }}
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            padding: 40px 20px;
        }}
        
        .header {{
            text-align: center;
            margin-bottom: 50px;
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(20px);
            border-radius: 25px;
            padding: 40px;
            box-shadow: 0 25px 50px rgba(0, 0, 0, 0.2);
        }}
        
        .title {{
            font-size: 3.5rem;
            font-weight: 800;
            margin-bottom: 15px;
            background: linear-gradient(45deg, #667eea, #764ba2, #f093fb);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
        }}
        
        .subtitle {{
            font-size: 1.3rem;
            color: #666;
            margin-bottom: 30px;
        }}
        
        .top-matches {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(400px, 1fr));
            gap: 30px;
            margin-bottom: 50px;
        }}
        
        .character-card {{
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(20px);
            border-radius: 25px;
            padding: 30px;
            box-shadow: 0 25px 50px rgba(0, 0, 0, 0.15);
            transition: all 0.3s ease;
            position: relative;
            overflow: hidden;
        }}
        
        .character-card:hover {{
            transform: translateY(-10px);
            box-shadow: 0 35px 70px rgba(0, 0, 0, 0.25);
        }}
        
        .character-card.rank-1 {{
            border: 3px solid #ffd700;
            background: linear-gradient(135deg, rgba(255, 215, 0, 0.1) 0%, rgba(255, 255, 255, 0.95) 100%);
        }}
        
        .character-card.rank-2 {{
            border: 3px solid #c0c0c0;
            background: linear-gradient(135deg, rgba(192, 192, 192, 0.1) 0%, rgba(255, 255, 255, 0.95) 100%);
        }}
        
        .character-card.rank-3 {{
            border: 3px solid #cd7f32;
            background: linear-gradient(135deg, rgba(205, 127, 50, 0.1) 0%, rgba(255, 255, 255, 0.95) 100%);
        }}
        
        .rank-badge {{
            position: absolute;
            top: -10px;
            right: -10px;
            width: 60px;
            height: 60px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 1.5rem;
            font-weight: 800;
        }}
        
        .rank-1 .rank-badge {{
            background: linear-gradient(135deg, #ffd700, #ffed4e);
            color: #333;
        }}
        
        .rank-2 .rank-badge {{
            background: linear-gradient(135deg, #c0c0c0, #e8e8e8);
            color: #333;
        }}
        
        .rank-3 .rank-badge {{
            background: linear-gradient(135deg, #cd7f32, #deb887);
            color: white;
        }}
        
        .character-image {{
            width: 150px;
            height: 200px;
            object-fit: cover;
            border-radius: 15px;
            margin: 0 auto 20px;
            display: block;
            box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2);
        }}
        
        .character-name {{
            font-size: 1.8rem;
            font-weight: 700;
            text-align: center;
            margin-bottom: 15px;
            color: #333;
        }}
        
        .match-score {{
            text-align: center;
            margin-bottom: 20px;
        }}
        
        .score-value {{
            font-size: 2.5rem;
            font-weight: 800;
            color: #667eea;
        }}
        
        .score-label {{
            font-size: 0.9rem;
            color: #666;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        
        .character-summary {{
            background: rgba(102, 126, 234, 0.1);
            border-radius: 15px;
            padding: 20px;
            margin-top: 20px;
            font-size: 0.95rem;
            line-height: 1.5;
            color: #555;
        }}
        
        .all-results {{
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(20px);
            border-radius: 25px;
            padding: 40px;
            box-shadow: 0 25px 50px rgba(0, 0, 0, 0.15);
            margin-top: 50px;
        }}
        
        .results-title {{
            font-size: 2.2rem;
            font-weight: 700;
            text-align: center;
            margin-bottom: 30px;
            color: #333;
        }}
        
        .result-item {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 15px;
            margin-bottom: 10px;
            background: linear-gradient(135deg, #f8f9ff 0%, #e8f0ff 100%);
            border-radius: 10px;
            border-left: 4px solid #667eea;
        }}
        
        .result-rank {{
            font-weight: 600;
            color: #667eea;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1 class="title">🎭 Your Ideal Match Results</h1>
            <p class="subtitle">Based on your personality preferences and compatibility analysis</p>
            <p style="color: #888; margin-top: 15px;">Analysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        </div>
        
        <div class="top-matches">
"""
    
    # Add top 3 character cards
    crowns = ["👑", "🥈", "🥉"]
    for char in top_3_details:
        rank_class = f"rank-{char['rank']}"
        html_content += f"""
            <div class="character-card {rank_class}">
                <div class="rank-badge">{crowns[char['rank']-1]}</div>
                <img src="{char['image']}" alt="{char['name']}" class="character-image" 
                     onerror="this.style.display='none';">
                
                <h3 class="character-name">{char['name']}</h3>
                
                <div class="match-score">
                    <div class="score-value">{char['score']:.1f}%</div>
                    <div class="score-label">Match Score</div>
                </div>
                
                <div class="character-summary">
                    {char['summary']}
                </div>
            </div>
"""
    
    # Add complete results section
    html_content += f"""
        </div>
        
        <div class="all-results">
            <h2 class="results-title">📊 Complete Rankings</h2>
"""
    
    # Add all character results
    for i, (name, score, _) in enumerate(top_matches, 1):
        emoji = "👑" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "💫"
        html_content += f"""
            <div class="result-item">
                <span><span style="font-size: 1.2rem; margin-right: 10px;">{emoji}</span>#{i} {name}</span>
                <span class="result-rank">{score:.1f}%</span>
            </div>
"""
    
    html_content += """
        </div>
    </div>
</body>
</html>
"""
    
    return html_content

def display_beautiful_results(top_matches, user_profile):
    """Display results in a beautiful web page"""
    html_content = create_beautiful_results_html(top_matches, user_profile)
    
    with tempfile.NamedTemporaryFile(mode='w', suffix='.html', delete=False, encoding='utf-8') as f:
        f.write(html_content)
        html_file = f.name
    
    try:
        webbrowser.open('file://' + html_file)
        print("🌟 Beautiful results page opened in your browser!")
    except Exception as e:
        print(f"❌ Error opening browser: {e}")
    
    print(f"💾 Results saved to: {html_file}")
    print("📌 You can bookmark this file to keep your results!")
    
    return html_file

print("🌟 Beautiful results generator ready!")

🌟 Beautiful results generator ready!


## 🎯 **Ready to Find Your Ideal Match!**

**Run the cell below to start your personality assessment:**

- ✨ **7 engaging scenarios** to test your preferences
- 🎭 **Interactive choices** with multiple preference levels
- 🏆 **Beautiful results** with character images and detailed analysis
- 📊 **Complete rankings** of all characters

---

In [8]:
# 🚀 MAIN ASSESSMENT - Run This to Start!

def run_ideal_match_assessment():
    """Main function to run the complete assessment"""
    
    print("🎭 Welcome to the Ideal Anime Character Match Predictor!")
    print("=" * 70)
    print("🌟 You're about to discover your perfect anime character match!")
    print("📊 Based on advanced personality compatibility analysis")
    print("=" * 70)
    
    # Initialize assessment variables
    user_preferences = pd.Series(0.0, index=trait_cols)
    scenario_history = []
    MAX_QUESTIONS = 7
    
    print(f"\n🎉 Starting Interactive Assessment ({MAX_QUESTIONS} scenarios)")
    print("💡 Answer honestly for the most accurate results!")
    print("=" * 70)
    
    # Main assessment loop
    for q_num in range(1, MAX_QUESTIONS + 1):
        print(f"\n--- Scenario {q_num}/{MAX_QUESTIONS} ---")
        
        # Get a unique scenario
        scenario = get_scenario(history=scenario_history)
        scenario_history.append(scenario['scenario_question'])
        
        # Get user choice
        multiplier, choice = ask_scenario_question(scenario, q_num, MAX_QUESTIONS)
        
        # Update user preferences based on choice
        if multiplier > 0:
            chosen_vector_key = "vector_a" if choice == "a" else "vector_b"
            score_vector = scenario[chosen_vector_key]
            print(f"  📊 Your choice updated these traits: {list(score_vector.keys())}")
            
            for trait, score in score_vector.items():
                if trait in user_preferences:
                    user_preferences[trait] += score * multiplier
        else:
            print(f"  ➡️  Neutral choice - no profile changes")
        
        time.sleep(0.3)  # Brief pause for readability
    
    # Calculate compatibility scores
    print("\n" + "=" * 70)
    print("🔮 Calculating compatibility with all characters...")
    print("=" * 70)
    
    # Use dot product for similarity calculation
    match_scores = traits_norm.values @ user_preferences.values
    
    # Create results dataframe
    rank_df = pd.DataFrame({
        NAME_COL: df[NAME_COL],
        SUMMARY_COL: df[SUMMARY_COL],
        "match_score": match_scores
    })
    rank_df = rank_df.sort_values("match_score", ascending=False).reset_index(drop=True)
    
    # Prepare data for beautiful display
    top_matches_list = []
    
    for i, row in rank_df.iterrows():
        char_name = row[NAME_COL]
        raw_score = row['match_score']
        # Convert to percentage (normalize between 0-100)
        match_percentage = max(0, min(100, (raw_score + 0.5) * 100))
        
        char_data = {
            'summary': row[SUMMARY_COL],
            'raw_score': raw_score,
            'personality_score': raw_score,
        }
        
        top_matches_list.append((char_name, match_percentage, char_data))
    
    # Display results
    print("\n" + "=" * 70)
    print("🎉 ANALYSIS COMPLETE! 🎉")
    print("=" * 70)
    print("🌟 Opening beautiful results in your browser...")
    
    # Create and display beautiful web results
    results_file = display_beautiful_results(top_matches_list, user_preferences.to_dict())
    
    # Console results summary
    print("\n" + "=" * 70)
    print("🏆 YOUR TOP 5 MATCHES! 🏆")
    print("=" * 70)
    
    top_5 = rank_df.head(5)
    rank_emojis = ["🥇", "🥈", "🥉", "🏅", "🏅"]
    
    for i, row in top_5.iterrows():
        match_percentage = max(0, min(100, (row['match_score'] + 0.5) * 100))
        
        print(f"\n{rank_emojis[i]} #{i+1}: {row[NAME_COL]}")
        print(f"   💖 Match Score: {match_percentage:.1f}%")
        print(f"   📖 {row[SUMMARY_COL]}")
        print("-" * 50)
    
    # Show personality insights
    significant_preferences = user_preferences[abs(user_preferences) > 0.1].sort_values(ascending=False)
    print(f"\n🎯 Your personality preferences:")
    if len(significant_preferences) > 0:
        for trait, score in significant_preferences.head(5).items():
            direction = "↗️" if score > 0 else "↘️"
            print(f"   {direction} {trait}: {score:.2f}")
    else:
        print("   🌟 A balanced personality - you're open to many types!")
    
    print(f"\n💡 Your ideal match is: **{rank_df.iloc[0][NAME_COL]}**!")
    print("=" * 70)
    print("🌐 Check your browser for the complete beautiful results!")
    print("=" * 70)
    
    return rank_df, user_preferences, results_file

# Run the assessment!
print("🎯 Ready to find your ideal anime character match!")
print("▶️  Execute this cell to start the assessment!")
print()
print("💡 Uncomment the line below to begin:")
print("# results, preferences, html_file = run_ideal_match_assessment()")
print()
print("🚀 Or run it directly:")

# Uncomment the line below to start the assessment:
results, preferences, html_file = run_ideal_match_assessment()

🎯 Ready to find your ideal anime character match!
▶️  Execute this cell to start the assessment!

💡 Uncomment the line below to begin:
# results, preferences, html_file = run_ideal_match_assessment()

🚀 Or run it directly:
🎭 Welcome to the Ideal Anime Character Match Predictor!
🌟 You're about to discover your perfect anime character match!
📊 Based on advanced personality compatibility analysis

🎉 Starting Interactive Assessment (7 scenarios)
💡 Answer honestly for the most accurate results!

--- Scenario 1/7 ---

══════════════════════════════════════════════════════════════════════
✨ SCENARIO 1/7 ✨
══════════════════════════════════════════════════════════════════════
🎭 Facing a moral dilemma. Your partner:

🔵 A) A) Does what feels right in their heart
🔴 B) B) Analyzes consequences logically first

Choose your preference:
  💪 [1] Strongly A    👍 [2] Prefer A    🤷 [3] Neutral    👍 [4] Prefer B    💪 [5] Strongly B
══════════════════════════════════════════════════════════════════════


✅ You chose: 💪 Strongly prefer A
  📊 Your choice updated these traits: ['Empathy', 'Impulsiveness']

--- Scenario 2/7 ---

══════════════════════════════════════════════════════════════════════
✨ SCENARIO 2/7 ✨
══════════════════════════════════════════════════════════════════════
🎭 Training session with others. Your partner:

🔵 A) A) Pushes themselves to outperform everyone
🔴 B) B) Focuses on helping weaker teammates improve

Choose your preference:
  💪 [1] Strongly A    👍 [2] Prefer A    🤷 [3] Neutral    👍 [4] Prefer B    💪 [5] Strongly B
══════════════════════════════════════════════════════════════════════

--- Scenario 2/7 ---

══════════════════════════════════════════════════════════════════════
✨ SCENARIO 2/7 ✨
══════════════════════════════════════════════════════════════════════
🎭 Training session with others. Your partner:

🔵 A) A) Pushes themselves to outperform everyone
🔴 B) B) Focuses on helping weaker teammates improve

Choose your preference:
  💪 [1] Strongly A    👍 [2]

## 🎉 **Assessment Complete!**

### 🌟 **What You Get:**
- **Beautiful web page** with your top 3 matches and character images
- **Complete rankings** of all 26 characters
- **Personality insights** showing your preferences
- **Match percentages** based on scientific compatibility analysis

### 🔬 **How It Works:**
1. **Personality Assessment**: 7 scenarios test different personality dimensions
2. **Trait Scoring**: Your choices build a multi-dimensional preference profile
3. **Compatibility Calculation**: Advanced dot-product similarity with normalized character traits
4. **Beautiful Results**: Stunning web interface with character images and detailed analysis

### 🎯 **Features:**
- ✅ **20+ Personality Traits** analyzed
- ✅ **26 Anime Characters** from popular series
- ✅ **Interactive Console Interface** with emoji feedback
- ✅ **Beautiful HTML Results** with character images
- ✅ **Scientific Algorithm** using normalized trait vectors
- ✅ **Detailed Analytics** and personality insights

---

**🎭 Enjoy discovering your ideal anime character match!**